<a href="https://colab.research.google.com/github/imchihchao/aop113b/blob/main/ml_ptt_movies_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 爬取 PTT 電影版

In [1]:
import requests
from bs4 import BeautifulSoup
import csv

def get_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    cntt = soup.find('div', id='main-content')
    for tag in cntt.find_all(['span', 'div'], class_=['article-metaline','article-metaline-right','push','f2']):
        tag.decompose()
    cleaned_content = cntt.get_text(strip=True)
    return cleaned_content

def get_articles(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    for atcl in soup.find_all('div', class_='r-ent'):
        if atcl.find('div', class_='title').a:
            ttle = atcl.find('div', class_='title').a.text
            href = 'https://www.ptt.cc' + atcl.find('div', class_='title').a.get('href')
            date = atcl.find('div', class_='date').text
            cntt = get_article(href)
            articles.append([date, href, ttle, f'{ttle}\n{cntt}'])
    url = 'https://www.ptt.cc' + soup.find_all('a', class_='btn wide')[1].get('href')
    return url

articles = [['date','href','title','content']]

url = 'https://www.ptt.cc/bbs/Movie/index.html'
for i in range(5):
    url = get_articles(url)

with open('ptt_movies.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(articles)

In [2]:
import pandas as pd
df = pd.read_csv('ptt_movies.csv')
df

,date,href,title,content
0,6/01,https://www.ptt.cc/bbs/movie/M.1748791058.A.90...,[好無雷] ULTRA4DX 名偵探柯南 獨眼的殘像,[好無雷] ULTRA4DX 名偵探柯南 獨眼的殘像\nhttps://i.imgur.co...
1,6/02,https://www.ptt.cc/bbs/movie/M.1748793857.A.CF...,[討論] 看完不可能任務8一定要再看7,[討論] 看完不可能任務8一定要再看7\n防雷\n\n\n當初看7時一頭霧水\n看完8才知道...
2,6/02,https://www.ptt.cc/bbs/movie/M.1748800313.A.57...,Re: [雷] 《破地獄》香港觀眾的品味怎麼了？,Re: [雷] 《破地獄》香港觀眾的品味怎麼了？\n原文就有雷。\n\n但這篇主要是要釐清原...
3,6/02,https://www.ptt.cc/bbs/movie/M.1748820881.A.25...,[公告] 水桶,[公告] 水桶\n作者KKKKKvs333 (NOW_EMPiRE & Ayuni_D)\n...
4,6/02,https://www.ptt.cc/bbs/movie/M.1748825927.A.87...,[討論] 其實好萊塢還有很多題材可拍,[討論] 其實好萊塢還有很多題材可拍\n好萊塢還是有很多題材可以拍吧\n例如人魔，安東尼霍普...
...,...,...,...,...
88,5/30,https://www.ptt.cc/bbs/movie/M.1748535654.A.DF...,[討論] 不可能的任務系列似乎真的畫下句點？,[討論] 不可能的任務系列似乎真的畫下句點？\n剛剛阿湯哥發了一系列的MI花絮照\n \n分...
89,5/30,https://www.ptt.cc/bbs/movie/M.1748536934.A.6D...,[討論] 花木蘭，迪士尼拍最好的政治正確電影,[討論] 花木蘭，迪士尼拍最好的政治正確電影\nhttps://i.imgur.com/QO...
90,5/30,https://www.ptt.cc/bbs/movie/M.1748539251.A.EA...,[問片] 約1995左右有碎木機的驚悚片(有雷),[問片] 約1995左右有碎木機的驚悚片(有雷)\n因此觀影時可能不到十歲，所以只記得結尾部...
91,5/30,https://www.ptt.cc/bbs/movie/M.1748539280.A.1A...,[新聞]《不可能的任務》本該大放異彩的8位遺珠！,[新聞]《不可能的任務》本該大放異彩的8位遺珠！\n雷\n\n\n\n\n\n\n\n\n\...


## 每筆討論的情感分析

In [8]:
from google.colab import userdata
from google import genai
import time

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

id = 0
def analyze_sentiment(row):
    global id
    prompt = f'''
    請你扮演一位具有語意分析能力的文字分析師，針對以下社交媒體上關於電影的討論內容，進行整體風向的判斷。請依據文字內容判定觀點為正面、負面或中性，並簡要說明你做出此判斷的依據。

    請用 JSON 格式 回覆，包含以下三個欄位：
    1."movie"：電影名稱（若無明確指出，請填入 "unknown"）
    2."sentiment"：整體風向（"positive"、"negative" 或 "neutral"）
    3."reasoning"：你根據哪些內容判斷出該風向，簡要說明理由

    以下是需要分析的討論內容：
    {row}
    '''
    while True:
        time.sleep(1)
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash-lite",
                contents=[prompt]
            )
            id += 1
            return response.text
        except Exception as e:
            print(f"ID: {id}; Error generating content: {e}.")
            time.sleep(2)

In [9]:
df['sentiment'] = df['content'].apply(analyze_sentiment)

In [10]:
df

,date,href,title,content,sentiment
0,6/01,https://www.ptt.cc/bbs/movie/M.1748791058.A.90...,[好無雷] ULTRA4DX 名偵探柯南 獨眼的殘像,[好無雷] ULTRA4DX 名偵探柯南 獨眼的殘像\nhttps://i.imgur.co...,"```json\n{\n ""movie"": ""名偵探柯南 獨眼的殘像"",\n ""sent..."
1,6/02,https://www.ptt.cc/bbs/movie/M.1748793857.A.CF...,[討論] 看完不可能任務8一定要再看7,[討論] 看完不可能任務8一定要再看7\n防雷\n\n\n當初看7時一頭霧水\n看完8才知道...,"```json\n{\n ""movie"": ""不可能的任務8"",\n ""sentimen..."
2,6/02,https://www.ptt.cc/bbs/movie/M.1748800313.A.57...,Re: [雷] 《破地獄》香港觀眾的品味怎麼了？,Re: [雷] 《破地獄》香港觀眾的品味怎麼了？\n原文就有雷。\n\n但這篇主要是要釐清原...,"```json\n{\n ""movie"": ""破地獄"",\n ""sentiment"": ..."
3,6/02,https://www.ptt.cc/bbs/movie/M.1748820881.A.25...,[公告] 水桶,[公告] 水桶\n作者KKKKKvs333 (NOW_EMPiRE & Ayuni_D)\n...,"```json\n{\n ""movie"": ""不可能的任務8"",\n ""sentimen..."
4,6/02,https://www.ptt.cc/bbs/movie/M.1748825927.A.87...,[討論] 其實好萊塢還有很多題材可拍,[討論] 其實好萊塢還有很多題材可拍\n好萊塢還是有很多題材可以拍吧\n例如人魔，安東尼霍普...,"```json\n{\n ""movie"": ""unknown"",\n ""sentimen..."
...,...,...,...,...,...
88,5/30,https://www.ptt.cc/bbs/movie/M.1748535654.A.DF...,[討論] 不可能的任務系列似乎真的畫下句點？,[討論] 不可能的任務系列似乎真的畫下句點？\n剛剛阿湯哥發了一系列的MI花絮照\n \n分...,"```json\n{\n ""movie"": ""不可能的任務系列 (Mission: Imp..."
89,5/30,https://www.ptt.cc/bbs/movie/M.1748536934.A.6D...,[討論] 花木蘭，迪士尼拍最好的政治正確電影,[討論] 花木蘭，迪士尼拍最好的政治正確電影\nhttps://i.imgur.com/QO...,"```json\n{\n ""movie"": ""花木蘭"",\n ""sentiment"": ..."
90,5/30,https://www.ptt.cc/bbs/movie/M.1748539251.A.EA...,[問片] 約1995左右有碎木機的驚悚片(有雷),[問片] 約1995左右有碎木機的驚悚片(有雷)\n因此觀影時可能不到十歲，所以只記得結尾部...,"```json\n{\n ""movie"": ""unknown"",\n ""sentimen..."
91,5/30,https://www.ptt.cc/bbs/movie/M.1748539280.A.1A...,[新聞]《不可能的任務》本該大放異彩的8位遺珠！,[新聞]《不可能的任務》本該大放異彩的8位遺珠！\n雷\n\n\n\n\n\n\n\n\n\...,"```json\n{\n ""movie"": ""不可能的任務"",\n ""sentiment..."


In [15]:
import json
for id, row in df.iterrows():
    try:
        js = json.loads(row['sentiment'].replace('```json','').replace('```',''))
        print(f'ID: {id}')
        print(f'電影名稱: {js["movie"]}；情感分析： {js["sentiment"]}')
        print(js['reasoning'])
    except:
        print(f'ID: {id}')
        print(row['sentiment'])

ID: 0
電影名稱: 名偵探柯南 獨眼的殘像；情感分析： positive
雖然指出了一些缺點，例如犯人動機稍嫌牽強，4DX效果不如預期，以及部分對白難以理解，但整體上給予正面評價。 觀影者表示運氣好訂到票，認為能看到毛利小五郎耍帥很值得，眾人出場機率平均，且表明回台灣還會再看一次，顯示對電影的喜愛，且最後一句「能沉浸並將自己代入在作品之中，那才是好的作品」也透露著正面評價。
ID: 1
電影名稱: 不可能的任務8；情感分析： positive
雖然討論中提到觀看7時感到困惑，但整體而言，討論者認為觀看8可以理解7中的伏筆，並在重看7後發現其精彩之處。 討論者用了「就像大話西遊那樣」、「意外的發現看懂也非常好看了」等正面的表達，表示對電影的理解和欣賞。
ID: 2
電影名稱: 破地獄；情感分析： positive
該評論者針對另一篇負面評論進行反駁，並詳細分析了電影劇情，指出原評論者對電影的理解有偏差。 評論者認為電影的重點在於探討生人面對的困境，以及父愛，並對電影的轉折和深度給予高度評價，認為電影超越了《送行者》。評論者整體上對電影給予了正面的評價，認為是一部值得一看的作品，並給出了90分的高分。
ID: 3
電影名稱: 不可能的任務8；情感分析： negative
雖然文章開頭提到電影內容並給予了正面評價，但隨後大量篇幅轉向批評特定國家的總統及其支持者，表達了強烈的不滿。由於主要內容集中在負面情緒的表達，且發文者因言論違反版規而遭到水桶，因此整體風向判定為負面。
ID: 4
電影名稱: unknown；情感分析： neutral
討論內容提出好萊塢可以嘗試的題材，並未直接批評或稱讚任何電影，主要在於建議與討論，偏向中性立場。雖然提到了對編劇的期望，但語氣溫和，不帶明顯的情緒色彩。
ID: 5
電影名稱: 闇黑電話2；情感分析： positive
文章提到《闇黑電話》首集票房不錯，並且續集由原班人馬回歸，包括導演、男主角和演員伊森霍克，這些都指向了正面評價。此外，續集是原創劇情，也可能吸引觀眾。
ID: 6
電影名稱: 瘋狂黑澀會 (推測)；情感分析： neutral
該討論內容主要是尋求電影名稱，表達了對電影的懷念和希望再次觀看的願望。雖然描述了電影的劇情片段，但並未包含對電影品質的評價，因此整體風向為中性。
ID: 7
電影名稱: 不可能的任務8；情感分析：

## 整體風向分析

In [18]:
from google.colab import userdata
from google import genai
from IPython.display import display, Markdown

client = client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

prompt = f'''
請你針對以下社群留言內容進行分析，完成以下任務：

1.摘要出留言的主要重點，找出大家關注的核心議題或觀點。
2.辨識並補充每個重點中表現出來的情緒（如正面、負面或中性，以及具體的情緒如憤怒、喜悅、失望等）。
3.綜合分析留言中最多人表現出的主情緒及其對應的討論內容，並說明整體發文風向（例如：偏正面、偏負面或中立），以及你做出此判斷的理由。

請用條列方式呈現分析結果，結尾請明確指出整體風向。以下是留言內容：
{df.to_string()}
'''

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[prompt]
)

display(Markdown(response.text))

好的，我來分析社群留言內容，並依據你的要求條列式呈現結果。

**1. 主要重點摘要：**

*   **名偵探柯南 獨眼的殘像：** 觀影體驗分享，提及 4DX 效果普通，但毛利小五郎的表現值得一看，犯人動機牽強。
*   **不可能的任務8：** 需搭配第7集觀看才能理解劇情伏筆，重看第7集後發現其精彩之處。另有影評質疑劇情不合理，過於英雄主義。也有討論總統的小孩有當兵而引發的政治話題。
*   **破地獄：** 影評對電影評價兩極，有人認為平庸、劇本割裂，有人則反駁並認為電影連貫，探討生人困境與父愛。
*   **好萊塢題材：** 討論好萊塢電影題材的選擇，建議可以嘗試人魔、男人消失等題材。
*   **闇黑電話2：** 續集預告釋出，由原班人馬打造，劇情為全新原創。
*   **韓國電影尋片：** 尋找一部台灣翻譯為「瘋狂黑澀會」的韓國搞笑電影。
*   **黃衣小飛俠：** 觀影心得，認為編劇將山林傳說還原得不錯，劇情編排有意料之外的內容。
*    **關於愛的練習：** 觀影後覺得這部片很好看。
*    **《國有器官》全球播映：** 呼籲共同制止中中共活摘器官，應關注人權問題。
*   **迷走星球：** 詢問劇情細節，稱讚女演員外貌。
*   **搜查瑠公圳：** 討論政治戲份與角色關係，認為整體來說蠻好看的。
*   **室友梅根：** 對 AI 玩偶梅根的設定進行討論，表示想要一個，但希望不要太恐怖。
*    **膽大黨 斜視：** 雖然結尾略吊胃口，但整體評論偏正面。
*    **新幹線驚爆倒數的疑問：** 對電影情節細節提出疑問。
*   **亞莉安娜加入《門當父不對》第四集：** 報導新卡司加入，並公布上映日期。
*   **高爾夫球也瘋狂 2：** 正式預告釋出。
*   **死亡筆記本 (美版)：** 負評，認為被改成一坨大便，與原作相差甚遠。
*   **新幹線驚爆倒數：** 對劇情邏輯的合理性提出質疑。
*   **史迪奇真人版：** 討論結局改編，認為更符合現代價值觀，但對部分改編不滿。
*    **滿血復活：** 導演大受好評，漫威影業、DC 影業都已經找上導演檔洽談合作。
*    **迷走星球：** 詢問劇情細節，稱讚女演員外貌。
*   **Live 文：** 分享各個電影台將播放的電影。
*   **廖俊是否為大咖：** 討論早期台灣電影演員廖俊的地位。
*   **為何現在戰爭片沒以前經典：** 認為現在戰爭片不如以往。
*   **禁政治版規：** 公告修改的版規及投票結果。

**2. 情緒辨識與補充：**

*   **名偵探柯南 獨眼的殘像：** 正面（好奇、值得、難以認同）。
*   **不可能的任務8：** 正面（欣賞）、負面（困惑、不合理、不滿）。
*   **破地獄：** 正面（讚賞、高度評價）、負面（失望、震驚、水準不符）。
*   **好萊塢題材：** 中性（建議、期望）。
*   **闇黑電話2：** 正面（期待）。
*   **韓國電影尋片：** 中性（懷念、希望）。
*   **黃衣小飛俠：** 正面（期待、滿意、推薦）。
*    **關於愛的練習：** 正面（喜歡、感動、渴望）。
*    **《國有器官》全球播映：** 正面（支持、呼籲、希望）。
*   **迷走星球：** 正面（稱讚外貌）、中性（疑問）。
*   **搜查瑠公圳：** 正面（好看）、負面（疑問、批評演技）。
*   **室友梅根：** 正面（想要）、中性（希望不要太恐怖）。
*    **膽大黨 斜視：** 中性 (正面評價和負面描述互相抵銷）。
*    **新幹線驚爆倒數的疑問：** 中性（疑問）。
*   **亞莉安娜加入《門當父不對》第四集：** 正面（期待、樂觀）。
*   **高爾夫球也瘋狂 2：** 中性（資訊）。
*   **死亡筆記本 (美版)：** 負面（不滿、質疑、否定）。
*   **新幹線驚爆倒數：** 負面 (對情節產生疑惑與猜疑）。
*   **史迪奇真人版：** 正面（肯定、享受）、負面（不滿）。
*    **滿血復活：** 正面(期待）
*   **迷走星球：** 正面（稱讚外貌）、中性（疑問）。
*   **Live 文：** 中性（告知）。
*   **廖俊是否為大咖：** 中性（提問、猜測）。
*   **為何現在戰爭片沒以前經典：** 負面（失望、惋惜）。
*   **禁政治版規：** 負面（不滿、對規則有異議）。
*    ** 搜查瑠公圳：**正面（喜歡）、負面（失望）
*   **水桶公告：** 負面 (對於版規的不滿）。

**3. 主情緒分析與發文風向：**

*   **最多人表現出的主情緒：** 正面和負面情緒並存，但以**正面情緒**略勝一籌。許多留言表達對特定電影的喜愛、期待、讚賞或推薦。
*   **對應的討論內容：** 這些正面情緒主要集中在對電影的觀影體驗、演員表現、劇情改編、以及對特定系列作品的期待。與《不可能的任務8》相關的討論中，雖有部分負面評價，但也有許多正面迴響。此外，如《她眼中的世界》、《關於愛的練習》等電影也獲得了正面評價。
*   **整體發文風向：** 偏向**中性偏正面**。 理由是雖然有負面情緒的發言，但多數留言在分享資訊、提問、或表達個人感受，而正面評價的數量略高於負面評價，且許多討論貼文最終都以推薦或對電影的肯定作結。

希望這個分析對你有幫助！
